In [ ]:
pip install -r requirements.txt

In [ ]:
import skimage
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LightSource
from matplotlib import cm
from support import plot

#load image
z = skimage.io.imread('images/banana.png') / 255.0


In [ ]:
nrows, ncols, n_channels = z.shape
x = np.linspace(0., 1., ncols)
y = np.linspace(0., 1., nrows)
x, y = np.meshgrid(x, y)


# Set up plot
fig, ax = plt.subplots(subplot_kw=dict(projection='3d'))

bins = 8
img = np.digitize(z, np.linspace(0., 1., bins)) * 256 // bins
surf = ax.plot_surface(x, y, 256 - img[:,:,0], rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False, shade=True)

plt.show()

In [ ]:
plt.imshow(z[::20,::20, 0], cmap=cm.coolwarm)

In [ ]:
bins = 8
img = np.digitize(z[::20,::20, :], np.linspace(0., 1., bins)) * 256 // bins
plt.imshow(img)


In [ ]:
bins = np.linspace(0., 1., 16)
nums = np.linspace(0., 1., 1000)
plt.plot(nums, np.digitize(nums ** 2, bins))

In [ ]:
plt.imshow(z)

In [ ]:
z = skimage.io.imread('images/banana.png')
int_repr = z[:,:,0] * 2 ** 16 + z[:,:,1] * 2 ** 8 + z[:,:,2]
# convert uint array back to color channel bytes
byte_repr = np.zeros((nrows, ncols, 3), dtype=np.uint8)
int_repr += 100
byte_repr[:,:,0] = int_repr // 2 ** 16
byte_repr[:,:,1] = (int_repr % 2 ** 16) // 2 ** 8
byte_repr[:,:,2] = int_repr % 2 ** 8
plt.imshow(byte_repr)

In [ ]:
img = skimage.io.imread('images/banana.png')
img = skimage.color.rgba2rgb(img)
img = skimage.color.rgb2gray(img)
plt.imshow(img)

In [ ]:
plt.hist(np.reshape(img * 256, [-1]), bins=256, range=(0, 256), histtype='step', density=True)

In [ ]:
img2 = np.maximum(img - 0.5, 0)
plt.hist(np.reshape(img2 * 256, [-1]), bins=256, range=(0, 256), histtype='step', density=True)
plt.show()
plt.imshow(img2, cmap = 'grey', vmax=1.)

In [ ]:
img3 = img2 * 2
plt.hist(np.reshape(img3 * 256, [-1]), bins=256, range=(0, 256), histtype='step', density=True)
plt.show()
plt.imshow(img3, cmap = 'grey', vmax=1.)

In [ ]:
import skimage
import numpy as np
import numpy.typing as npt
from support.bounding_boxes import draw_bounding_boxes, create_bounding_boxes
import matplotlib.pyplot as plt
from skimage.morphology import erosion, dilation, disk

img = skimage.io.imread('images/gummibaerchen.jpg')

# scale image
img = skimage.transform.resize(img, (300, 400))

plt.imshow(img)
plt.show()

chroma = np.max(img, axis=2) - np.min(img, axis=2)

plt.imshow(chroma, cmap='gray')
plt.show()

hsv = skimage.color.rgb2hsv(img)

hue, s, v = np.transpose(hsv, (2, 0, 1))

cm = plt.cm.hsv

n, bins, patches = plt.hist(np.reshape(hue, (-1,)), 25)
for i, p in enumerate(patches):
    plt.setp(p, 'facecolor', cm(i/25.)) # notice the i/25
plt.show()

mask = s > 0.32
mask = dilation(mask, footprint = disk(2))

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(hue, cmap='hsv')
axes[0].imshow(np.where( np.expand_dims(mask, -1), [0,0,0,0], [0,0,0,255]), cmap='gray')

axes[1].imshow(s, cmap='gray')

axes[2].imshow(v, cmap='gray')
plt.show()


segmentation = skimage.measure.label(mask, background=0)
plt.imshow(segmentation, cmap='tab20')
plt.show()

rectangles = create_bounding_boxes(segmentation)

def assign_label_color(hue: npt.NDArray[float], saturation: npt.NDArray[float]) -> str:
    """
    Assigns a color label based on the hue and saturation values of an image.
    Parameters:
    hue (npt.NDArray[float]): An array of hue values.
    saturation (npt.NDArray[float]): An array of saturation values.
    Returns:
    str: The assigned color label ('orange', 'yellow', 'green', or 'red').
    """
    # only use pixels with high saturation
    mask = saturation > 0.5
    # get the hue values of the pixels in the rectangle
    colors = hue[mask]
    # white if no colors are found
    if len(colors) < len(saturation) * 0.1:
        return 'white'
    # get the most frequent hue value
    hue_value = np.bincount((colors * 50).astype(int)).argmax()
    # assign a color based on the hue value
    if hue_value <= 4:
        return 'orange'
    elif hue_value <= 6:
        return 'yellow'
    elif hue_value <= 10:
        return 'green'
    return 'red'

labels = []
for rect in rectangles:
    x, y, w, h = rect
    hue_rect = hue[y:y+h, x:x+w]
    saturation_rect = s[y:y+h, x:x+w]
    labels.append(assign_label_color(hue_rect, saturation_rect))

draw_bounding_boxes(rectangles, labels)

plt.imshow(img, cmap='tab20')
plt.show()

In [ ]:
plt.cm.hsv(0.7)